# Universal Configuration Widget Example

This notebook demonstrates how to use the Universal Configuration Widget in Jupyter notebooks to replace manual configuration blocks with interactive UI forms.

## Features

- **Universal Support**: Works with any configuration class inheriting from `BasePipelineConfig`
- **Interactive UI**: Embedded web interface with real-time validation
- **Enhanced Error Handling**: Field-specific Pydantic validation errors with visual highlighting
- **Auto-Population**: Pre-populates forms using base configuration data
- **Seamless Integration**: Drop-in replacement for manual configuration blocks

## Setup

First, let's set up the base configuration and import the necessary modules:

In [ ]:
# Import required modules
from cursus.core.base.config_base import BasePipelineConfig
from cursus.steps.configs.config_processing_step_base import ProcessingStepConfigBase
from cursus.api.config_ui import (
    create_jupyter_config_widget,
    create_jupyter_pipeline_config_widget,
    UniversalConfigWidgetWithServer
)

# Create base configuration
base_config = BasePipelineConfig(
    author="lukexie",
    bucket="example-bucket", 
    role="arn:aws:iam::123456789012:role/SageMakerRole",
    region="NA",
    service_name="AtoZ",
    pipeline_version="1.0.0",
    project_root_folder="example-project"
)

print("✅ Base configuration created successfully!")
print(f"Author: {base_config.author}")
print(f"Bucket: {base_config.bucket}")
print(f"Region: {base_config.region}")

## Example 1: Single Configuration Widget

Replace manual configuration blocks with interactive widgets:

In [ ]:
# Instead of manually creating ProcessingStepConfigBase:
# processing_step_config = ProcessingStepConfigBase.from_base_config(
#     base_config,
#     processing_step_name="data_processing",
#     # ... many more fields to configure manually
# )

# Use the interactive widget instead:
processing_widget = create_jupyter_config_widget(
    config_class_name="ProcessingStepConfigBase",
    base_config=base_config,
    height="700px"
)

processing_widget.display()

### Retrieve the Configuration

After completing the form above and clicking "Save Configuration", then "Get Configuration":

In [ ]:
# Get the configuration data from the widget
config_data = processing_widget.get_config()

if config_data:
    # Create the actual configuration instance
    processing_step_config = ProcessingStepConfigBase(**config_data)
    
    print("✅ ProcessingStepConfigBase created successfully!")
    print(f"Processing step name: {processing_step_config.processing_step_name}")
    print(f"Number of fields configured: {len(config_data)}")
    
    # Add to your config list
    config_list = [base_config, processing_step_config]
    print(f"\n📋 Config list now has {len(config_list)} configurations")
else:
    print("❌ No configuration data available. Please complete the form above first.")

## Example 2: Multiple Configuration Types

Create widgets for different configuration types:

In [ ]:
# Create widget for Cradle Data Load Configuration
cradle_widget = create_jupyter_config_widget(
    config_class_name="CradleDataLoadConfig",
    base_config=base_config,
    height="800px"
)

cradle_widget.display()

In [ ]:
# Retrieve Cradle configuration
cradle_config_data = cradle_widget.get_config()

if cradle_config_data:
    from cursus.steps.configs.config_cradle_data_loading_step import CradleDataLoadConfig
    cradle_config = CradleDataLoadConfig(**cradle_config_data)
    
    print("✅ CradleDataLoadConfig created successfully!")
    print(f"Job type: {cradle_config.job_specification.job_type}")
    print(f"Data sources: {len(cradle_config.data_sources_specification.data_sources)}")
    
    # Add to config list
    config_list.append(cradle_config)
    print(f"\n📋 Config list now has {len(config_list)} configurations")
else:
    print("❌ No Cradle configuration data available. Please complete the form above first.")

## Example 3: Widget with Server Management

For cases where you need the widget to manage its own server:

In [ ]:
# Create widget that can start/stop its own server
server_widget = UniversalConfigWidgetWithServer(
    config_class_name="ModelHyperparameters",
    base_config=base_config,
    height="600px",
    server_port=8004
)

# This will automatically start the server if needed
server_widget.display()

## Key Features Demonstrated

### 1. Enhanced Error Handling
- Field-specific Pydantic validation errors
- Visual highlighting of invalid fields
- Auto-scroll to first error field
- User-friendly error messages

### 2. Real-time Validation
- 300ms debounced validation
- Type-specific validation rules
- JSON validation for complex fields

### 3. Enhanced User Experience
- Request deduplication and caching
- Loading state indicators
- Unsaved changes protection
- Form state management

### 4. Package Portability
- Proper relative imports
- Flexible deployment options
- Production-ready implementation